In [94]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a05_unidades_de_apoyo as ua
from proyeccion_rdr.produccion.a04_ambulatorio import (
    leer_consultas_medicas
)


pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

## Leer insumos de Apoyo Diagnostico

In [96]:
# Lee la proyeccion de consultas medicas
RUTA_CONSULTAS = "../data/interim/3.0_estimacion_boxes_medicos_RDR.xlsx"
consultas_medicas_proyectadas = features.leer_proyeccion_consultas_medicas_especialidad(
    RUTA_CONSULTAS, ANIOS_POBLACION
)

# Lee la cantidad de DCO proyectados
RUTA_DCO = "../data/interim/1.0_estimacion_atencion_cerrada_RDR.xlsx"
dco_proyectados = features.leer_dco_proyectados(RUTA_DCO, ANIOS_POBLACION)

In [97]:
# Define las rutas de los archivos
ruta_raw = "../data/raw/"

# Define las rutas de la base de CAE y GRD
ruta_cae = ruta_raw + "6_ambulatorio/df_procesada_consultas.csv"
ruta_grd = ruta_raw + "7_egresos_grd/df_procesada_grd.csv"
ruta_urgencias = ruta_raw + "8_urgencias/df_procesada_urgencias.csv"

# Define las rutas de los archivos de apoyo diagnostico
ruta_apoyo = ruta_raw + "9_unidades_de_apoyo/"

ruta_laboratorio = ruta_apoyo + "df_procesada_laboratorio_clinico.csv"
ruta_farmacia = ruta_farmacia_amb = ruta_apoyo + "df_procesada_farmacia.csv"
ruta_imagenologia = ruta_apoyo + "df_procesada_imagenologia.csv"
ruta_umt = ruta_apoyo + "df_procesada_umt.csv"

In [98]:
# Lee CAE y GRD
consultas_medicas = leer_consultas_medicas(ruta_cae)
df_grd = ua.leer_grd(ruta_grd)

C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a04_ambulatorio.py:90: DtypeWarning: Columns (10,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})
C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a05_unidades_de_apoyo.py:12: DtypeWarning: Columns (47,48,49,50,51,52,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})


In [99]:
# Lee las unidades de apoyo
df_laboratorio, df_laboratorio_hosp, df_laboratorio_amb = ua.leer_laboratorio(ruta_laboratorio)
df_farmacia, df_farmacia_hosp, df_farmacia_amb = ua.leer_farmacia(ruta_farmacia)

In [100]:
# Define la agrupacion a calcular
AGRUPACION_EXAMENES = ["ano", "id_paciente"]

# Obtiene los dco totales y consultas por pacientes por anio
consultas_por_pacientes = consultas_medicas.groupby(AGRUPACION_EXAMENES)["fecha_atencion"].count()
dco_por_pacientes = df_grd.groupby(AGRUPACION_EXAMENES)["estancia"].sum()

# Obtiene los examanes por paciente de laboratorio
examenes_ppct_lab_hosp = df_laboratorio_hosp.groupby(AGRUPACION_EXAMENES)["examen"].count()
examenes_ppct_lab_amb = df_laboratorio_amb.groupby(AGRUPACION_EXAMENES)["examen"].count()

# Obtiene las prescripciones por pacientes de farmacia
prescr_ppct_farmacia_hosp = df_farmacia_hosp.groupby(AGRUPACION_EXAMENES)["nombre_articulo"].count()
prescr_ppct_farmacia_amb = df_farmacia_amb.groupby(AGRUPACION_EXAMENES)["nombre_articulo"].count()

In [101]:
# Obtiene los examenes por DCO y consultas para laboratorio
_, examenes_por_dco_lab = ua.calcular_examenes_por_atencion(
    dco_por_pacientes, examenes_ppct_lab_hosp, "estancia", "examen"
)
_, examenes_por_consulta_lab = ua.calcular_examenes_por_atencion(
    consultas_por_pacientes, examenes_ppct_lab_amb, "fecha_atencion", "examen"
)

# Obtiene los examenes por DCO y consultas para laboratorio
_, prescr_por_dco = ua.calcular_examenes_por_atencion(
    dco_por_pacientes, prescr_ppct_farmacia_hosp, "estancia", "nombre_articulo"
)
_, prescr_por_consulta = ua.calcular_examenes_por_atencion(
    consultas_por_pacientes, prescr_ppct_farmacia_amb, "fecha_atencion", "nombre_articulo"
)

## Laboratorio Clinico

En este apartado se quiere saber cuantos examenes de Laboratorio Clinico se haran

In [102]:
print("Laboratorio Clinico - AC - Examenes por DCO")
display(examenes_por_dco_lab)

print("Laboratorio Clinico - AA - Examenes por CME")
display(examenes_por_consulta_lab)

Laboratorio Clinico - AC - Examenes por DCO


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2017,6781.0,14336,5.906895,13.123988,0.0,0.000000,2.900000,8.000000,677.000000
2018,7050.0,15511,5.405699,8.059222,0.0,0.000000,2.750000,7.770833,148.000000
2019,7255.0,15704,5.869289,8.475399,0.0,0.000000,3.250000,8.174242,189.800000
2020,4008.0,11966,7.675083,11.139853,0.0,0.636364,4.500000,10.719925,319.000000
2021,4544.0,18269,7.702273,9.850032,0.0,1.000000,4.466667,11.000000,162.000000
2022,5505.0,21862,7.507892,8.870251,0.0,1.285714,4.714286,10.500000,134.571429
2023,6183.0,15579,6.596584,8.709555,0.0,1.000000,4.000000,9.000000,173.777778
2024,6342.0,15844,6.794908,8.613264,0.0,0.878472,4.333333,9.592857,139.000000


Laboratorio Clinico - AA - Examenes por CME


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2017,33139.0,8068,0.595854,1.692162,0.0,0.0,0.0,0.000000,52.0
2018,34236.0,8583,0.614689,1.705245,0.0,0.0,0.0,0.000000,24.0
2019,35828.0,8872,0.637850,1.760020,0.0,0.0,0.0,0.000000,27.0
2020,23673.0,6245,0.823679,2.334450,0.0,0.0,0.0,0.000000,29.0
2021,25635.0,8999,0.479853,1.219814,0.0,0.0,0.0,0.250000,18.5
2022,30494.0,9599,0.451171,1.135537,0.0,0.0,0.0,0.250000,18.5
2023,33188.0,9605,0.456769,1.135998,0.0,0.0,0.0,0.250000,16.0
2024,35259.0,9272,0.415474,1.106048,0.0,0.0,0.0,0.071429,15.0


Como se ve, la cantidad de examenes por CME es muy baja, por lo que se ocupara el estandar MINSAL.

In [103]:
# Define constantes
CANTIDAD_EXAMENES_AC_A_HACER = examenes_por_dco_lab["75%"].max()
CANTIDAD_EXAMENES_AA_A_HACER = 3 # Estandar MINSAL
CANTIDAD_EXAMENES_URG_A_HACER = 2.5 # Estandar MINSAL
metricas_para_proyectar_laboratorio = pd.DataFrame(
    {
        "examenes_por_dco": [CANTIDAD_EXAMENES_AC_A_HACER],
        "examenes_por_consulta": [CANTIDAD_EXAMENES_AA_A_HACER],
    }
)

# Obtiene le cantidad de examenes de laboratorio requeridos para atencion cerrada
examenes_laboratorio_ac = dco_proyectados * CANTIDAD_EXAMENES_AC_A_HACER

# Obtiene la cantidad de examenes de laboratorio requeridos para atencion abierta
examanes_laboratorio_aa = consultas_medicas_proyectadas * CANTIDAD_EXAMENES_AA_A_HACER

# Suma la cantidad de examenes totales
examenes_laboratorio_proyectados = examanes_laboratorio_aa + examenes_laboratorio_ac

In [104]:
# Muestra la produccion actual
produccion_actual_laboratorio = (
    df_laboratorio_amb.groupby("ano").size() + df_laboratorio_hosp.groupby("ano").size()
)

In [105]:
display(produccion_actual_laboratorio)
display(round(examenes_laboratorio_proyectados))

ano
2015    468546
2016    488454
2017    507761
2018    485015
2019    554398
2020    425257
2021    504242
2022    549272
2023    522606
2024    514917
dtype: int64

2017    1832284.0
2018    1876677.0
2019    1923031.0
2020    1968848.0
2021    1995563.0
2022    2012506.0
2023    2027414.0
2024    2042290.0
2025    2054822.0
2026    2062584.0
2027    2065347.0
2028    2064926.0
2029    2062151.0
2030    2058157.0
2031    2054737.0
2032    2050191.0
2033    2043621.0
2034    2034602.0
2035    2026586.0
dtype: float64

In [106]:
produccion_promedio_laboratorio = produccion_actual_laboratorio[
    produccion_actual_laboratorio.index != 2025
].mean()
crecimiento_laboratorio = examenes_laboratorio_proyectados["2035"] / produccion_promedio_laboratorio
print(f"Aumento de Laboratorio: {crecimiento_laboratorio:.2f} veces")

Aumento de Laboratorio: 4.04 veces
